## Description
### This Jupyter Notebook is used for obtaining descriptors values to curate a training dataset.

#### Workflow:
1. The training dataset in the form of a .csv file is read into a dataframe, followed by generating a Pearson Correlation Table to identify collinearity within the set of descriptors. 
2. Descriptors which are highly collinear are removed.
3. The training data is then standardized. Two features with the highest Pearson Coefficient with the target variable are chosen for classifier training. With two features, we are able to visualize the classifier with a decision boundary on a 2D map.
4. Best hyperparameters are obtained for each type of classifiers.
5. Decision boundaries are created and visualized.
6. Further training is conducted by using charge of ligand as an additional descriptor.

In [1]:
import re #Import RegEx
import os
import math
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
# Assign directory names
directory = os.getcwd()
spe_directory = directory + '\\spe'
opt_directory = directory + '\\opt'
steric_map_dir = directory + '\\steric_maps_nocoord_training'
directory

'C:\\Users\\George\\Desktop\\Research_UCLA\\CIC\\Computational Work\\Project-3-2\\intermediates\\training_set'

In [3]:
#Input excel table containing metal, atom_a and atom_b name
df = pd.read_csv('input.csv')   

xyz_match = ['X           Y           Z']
LUMO_match = ['Alpha virt. eigenvalues']
HOMO_match = ['Alpha  occ. eigenvalues']
nbo_match = ['Natural Population Analysis']
spe_match = ['SCF Done']


#List for constructing dataframe
df_name = []
df_metal = []
df_atom_a = []
df_atom_b = []
df_angle = []
df_metal_atoma_distance = []
df_metal_atomb_distance = []
df_metal_nbo = []
df_a_nbo = []
df_b_nbo = []
df_HOMO_LUMO = []
df_category = []
df_ligand = []
df_AB_NBO = []
df_MAMB_distance = []
df_spe = []
df_natoms = []

In [4]:
# Aims
# Extract descriptor from output file of spe.
# Extract geometric descriptors (bond angles, bond lengths) and electronic descriptors (nbo charge, HOMO-LUMO gap)
# Input needed: No. for the donor atoms (A and B) as well as metal atom (Pd)

# Bond distance: given cartesian coordinates, find distance between atoms
# Bond angle: given three atoms, find angle


for subdir,dirs,files in os.walk(spe_directory):                  # Loop over each directory, subdirectory and files
    for file in files:                                      # Loop over each file
        if any([file.endswith('.out')]):                    # If file is a .out file
            filename = os.path.join(subdir, file)       # Return path to file
            name = Path(filename).stem           # Extract filename from the end of path and return as a string
            
         
            for x in df['Filename']:                      # For each element in the "Filename" column of the input file
                if x == name:
                    match_file = df[df['Filename'] == name] 
                    print(match_file)
            
                    metal = match_file.iloc[0,1]                    # Extract metal, atom_a and atom_b numbers from input files
                    atom_a = match_file.iloc[0,2]
                    atom_b = match_file.iloc[0,3]
            
            
#             print(name, 'Metal: ',metal, 'Atom_A: ', atom_a, 'Atom_B: ', atom_b)

            mylines = []

            with open (filename, 'rt') as myfile:       # Open .out for reading text
                # myfile = myfile.read()                # Read the entire file to a string
                for myline in myfile:                    # For each line, stored as myline,
                    mylines.append(myline)               # add its contents to mylines list.


                # Find XYZ Coordinates

                for line in mylines:
                    if 'NAtoms' in line:
                        number_list = re.findall('-?\d*\.?\d+',line)            # get NAtoms value
                        natoms = int(number_list[0])
                        print(natoms)

                for line in mylines:
                    for phrase in xyz_match:                                # iterate through each phrases
                        if phrase in line:                                          # check if phrase is in line

                                                                                    # For loop for generating the XYZ coordinates
                            count = 0
                            xyz = []
                            line_number = mylines.index(line) + 2
                            while count < natoms:
                                count = count + 1
                                xyz.append(mylines[line_number])
                                line_number = line_number + 1
                            
#                             # Convert into xyz file 
                            

#                             with open('xyz.txt', 'w') as filehandle:                   # Save xyz coordinates (list) into a txt file
#                                 for listitem in xyz:
#                                     filehandle.write(listitem)

                x_coord = []
                y_coord = []
                z_coord = []
                atom_symbol =[]
                element = {"1":'H', "6":"C", "7": "N", "8":"O", "9":"F", "15":"P", "16":"S", "17":"Cl", "26":"Fe", "35":"Br", "46":"Pd"}
                
                # Generate XYZ file in .txt form, then find xyz coordinates for metal, atom_a and atom_b
                for line in xyz:
                    number_list = re.findall('-?\d*\.?\d+',line)
                    atom_number = int(number_list[0])
                    element_number = int(number_list[1])
                    atom_x = float(number_list[3])
                    atom_y = float(number_list[4])
                    atom_z = float(number_list[5])
                    
                    # Make xyz coord into .txt file for percent buried volume
                    x_coord.append(atom_x)
                    y_coord.append(atom_y)
                    z_coord.append(atom_z)
                    atom_symbol.append(element[str(element_number)])
                    name_xyz = name + '_xyz.txt'            
                    xyz_df = pd.DataFrame([atom_symbol,x_coord,y_coord,z_coord])
                    xyz_df = xyz_df.transpose()
#                     xyz_df.to_csv(name_xyz, header=False, index=False, sep = " ")

                    # Identify metal, atom_a and atom_b numbers and obtain their respective x,y,z coords
                    if atom_number == metal:
                        metal_x = atom_x
                        metal_y = atom_y
                        metal_z = atom_z
                        if element_number != 46:
                            print('Wrong metal atom number assignment ',filename) # Checking if metal atom was assigned correctly
                    elif atom_number == atom_a:
                        a_x = atom_x
                        a_y = atom_y
                        a_z = atom_z
                    elif atom_number == atom_b:
                        b_x = atom_x
                        b_y = atom_y
                        b_z = atom_z
                        
                
                    

                #Extract metal atom distance
                
                metal_atoma_distance = math.sqrt((metal_x - a_x) ** 2 + (metal_y - a_y) ** 2 + (metal_z - a_z) ** 2)    
                metal_atomb_distance = math.sqrt((metal_x - b_x) ** 2 + (metal_y - b_y) ** 2 + (metal_z - b_z) ** 2)

                metal_atoma_vector = [a_x - metal_x, a_y - metal_y, a_z - metal_z]
                metal_atomb_vector = [b_x - metal_x, b_y - metal_y, b_z - metal_z]

                dot_pdt = np.dot(metal_atoma_vector,metal_atomb_vector)
                    
                #Extract A-M-B angle by using arcos rule
                angle = int(math.degrees(math.acos (dot_pdt / ( metal_atoma_distance * metal_atomb_distance ))))         

#                 print('A-M-B Angle: ', angle)
#                 print('Metal-AtomA Distance: ', metal_atoma_distance)
#                 print('Metal-AtomB Distance: ', metal_atomb_distance)



                # Find HOMO - LUMO energy gap

                LUMO = []
                for phrase in LUMO_match:
                    for line in mylines:
                        if phrase in line:
                            LUMO.append(line)
                number_list = re.findall('-?\d*\.?\d+',LUMO[0])            # get value from line containing LUMO
                LUMO_energy = float(number_list[0])                         # get first value from the line, which represents the LUMO

                HOMO = []
                for phrase in HOMO_match:
                    for line in mylines:
                        if phrase in line:
                            HOMO.append(line)
                number_list = re.findall('-?\d*\.?\d+',HOMO[-1])            # get value from line containing LUMO
                HOMO_energy = float(number_list[-1])                        # get lsat value from line, which represents HOMO

                HOMO_LUMO = LUMO_energy - HOMO_energy                       # HOMO LUMO gap, in A.U.
#                 print('HOMO-LUMO Gap: ', HOMO_LUMO)

                # Find NBO charge

                for phrase in nbo_match:                                       # iterate through each phrases
                    for line in mylines:
                        if phrase in line:                                          # check if phrase is in line

                            count = 0
                            nbo_xyz = []
                            line_number = mylines.index(line) + 6
                            while count < natoms:
                                count = count + 1
                                nbo_xyz.append(mylines[line_number])
                                line_number = line_number + 1

                            with open('nbo_xyz.txt', 'w') as filehandle:                   # Save nbo_xyz  into a txt file
                                for listitem in nbo_xyz:
                                    filehandle.write(listitem)

                                                                                            # Extract NBO charge for metal, atom A and atom B
                for line in nbo_xyz:
                    number_list = re.findall('-?\d*\.?\d+',line)
                    atom_number = int(number_list[0])

                    if atom_number == metal:
                        metal_nbo = float(number_list[1])

                    elif atom_number == atom_a:
                        a_nbo = float(number_list[1])

                    elif atom_number == atom_b:
                        b_nbo = float(number_list[1])
                        
                # Find SPE 
                
                for phrase in spe_match:
                    for line in mylines:
                        if phrase in line:
                            spe = line.replace("SCF Done:  E(RM06) =  " , "")
                            l = len(spe)
                            spe = float(spe[:l-27])
                            df_spe.append(spe)
                            
                # Naming category
                
                if 'nocoord-cis' in name:
                    category = 'nocoord-cis'
                    df_category.append('nocoord-cis')
                    name = name.replace('-spe-nocoord-cis',"")
                elif 'cocoord-cis' in name:
                    category = 'cocoord-cis'
                    df_category.append('cocoord-cis')
                    name = name.replace('-spe-cocoord-cis',"")
                elif 'etcoord-cis' in name:
                    category = 'etcoord-cis'
                    df_category.append('etcoord-cis')
                    name = name.replace('-spe-etcoord-cis',"")
                elif 'nocoord' in name:
                    category = 'nocoord'
                    df_category.append('nocoord')
                    name = name.replace('-spe-nocoord',"")
                elif 'cocoord' in name:
                    category = 'cocoord'
                    df_category.append('cocoord')
                    name = name.replace('-spe-cocoord',"")
                elif 'etcoord' in name:
                    category = 'etcoord'
                    df_category.append('etcoord') 
                    name = name.replace('-spe-etcoord',"")
                else:
                    print('error, no category assigned')
                 
                print(name, category, natoms)
                df_name.append(name)
                df_metal.append(metal)
                df_atom_a.append(atom_a)
                df_atom_b.append(atom_b)
                df_angle.append(angle)
                df_metal_atoma_distance.append(metal_atoma_distance)
                df_metal_atomb_distance.append(metal_atomb_distance)
                df_metal_nbo.append(metal_nbo)
                df_a_nbo.append(a_nbo)
                df_b_nbo.append(b_nbo)
                df_HOMO_LUMO.append(HOMO_LUMO)
                df_AB_NBO.append(a_nbo-b_nbo)
                df_MAMB_distance.append(metal_atoma_distance-metal_atomb_distance)
                df_natoms.append(natoms)
#                 df_ligand.append(name[:-12])
                
                
data = {'Name': df_name,
        'Category' : df_category,
       'Metal': df_metal, 
       'Atom_A' : df_atom_a, 
       'Atom_B' : df_atom_b, 
       'A-M-B Angle' : df_angle, 
       'M-A Distance' : df_metal_atoma_distance, 
       'M-B Distance' : df_metal_atomb_distance, 
       'Metal_NBO' : df_metal_nbo, 
       'A_NBO' : df_a_nbo, 
       'B_NBO' : df_b_nbo, 
       'HOMO_LUMO' : df_HOMO_LUMO,
       'A-B_NBO' : df_AB_NBO,
       'MA_MB_Difference' : df_MAMB_distance,
       'SPE': df_spe}       
                
    
descriptor_data = pd.DataFrame(data)
descriptor_data = descriptor_data.sort_values('Category')
print(descriptor_data)
descriptor_data.to_csv("output_nbo.csv") 


checking_data = {'Name': df_name,
                 'Category' : df_category,
                 'NAtoms': df_natoms} 

checking_df = pd.DataFrame(checking_data)
checking_df.to_csv("checking_df.csv") 

            Filename  Metal   Atom_A  Atom_B
1  n-n-1-spe-etcoord       1      12      13
47
n-n-1 etcoord 47
            Filename  Metal   Atom_A  Atom_B
6  n-n-2-spe-etcoord       1      12      13
50
n-n-2 etcoord 50
             Filename  Metal   Atom_A  Atom_B
11  n-n-3-spe-etcoord       1      12      13
46
n-n-3 etcoord 46
             Filename  Metal   Atom_A  Atom_B
16  n-n-4-spe-etcoord       1      12      13
64
n-n-4 etcoord 64
              Filename  Metal   Atom_A  Atom_B
145  n-n-5-spe-etcoord       1       2      12
41
n-n-5 etcoord 41
              Filename  Metal   Atom_A  Atom_B
106  p-p-1-spe-etcoord      59       7       8
71
p-p-1 etcoord 71
              Filename  Metal   Atom_A  Atom_B
108  p-p-2-spe-etcoord       1       2       3
74
p-p-2 etcoord 74
              Filename  Metal   Atom_A  Atom_B
112  p-p-3-spe-etcoord      78      10      11
90
p-p-3 etcoord 90
              Filename  Metal   Atom_A  Atom_B
115  p-p-4-spe-etcoord      65      25      26
77
p-p

In [5]:
# Aims
# Extract gibbs free energy from output file of opt.

df_filename = []
df_gibbs = []
df_category = []
df_ligand = []

match_list = ['Thermal correction to Gibbs Free Energy']

rootdir = os.getcwd()                                       # Get current working directory

count = 0

for subdir,dirs,files in os.walk(opt_directory):                  # Loop over each directory, subdirectory and files
    for file in files:                                      # Loop over each file
        if any([file.endswith('.out')]):                    # If file is a .out file
            filename = os.path.join(subdir, file)       # Return path to file

            mylines = []

            with open (filename, 'rt') as myfile:       # Open .out for reading text
                # contents = myfile.read()                # Read the entire file to a string
                for myline in myfile:                    # For each line, stored as myline,
                    mylines.append(myline)               # add its contents to mylines.
                    
                for line in mylines:
                    if 'NAtoms' in line:
                        number_list = re.findall('-?\d*\.?\d+',line)            # get NAtoms value
                        natoms = int(number_list[0])


                for phrase in match_list:                                       # iterate through each phrases
                    for line in mylines:
                        if phrase in line:                                          # check if phrase is in line
                            name = Path(filename).stem
                            line = line.replace("Thermal correction to Gibbs Free Energy=         " , "")
                            l = len(line)
#                             line = line[:l-27]
                            gfe_corr = float(line)
                            df_gibbs.append(gfe_corr)
                            
                            if 'nocoord-cis' in name:
                                category = 'nocoord-cis'
                                df_category.append('nocoord-cis')
                                name = name.replace('-opt-nocoord-cis',"")
                            elif 'cocoord-cis' in name:
                                category = 'cocoord-cis'
                                df_category.append('cocoord-cis')
                                name = name.replace('-opt-cocoord-cis',"")
                            elif 'etcoord-cis' in name:
                                category = 'etcoord-cis'
                                df_category.append('etcoord-cis')
                                name = name.replace('-opt-etcoord-cis',"")
                            elif 'nocoord' in name:
                                category = 'nocoord'
                                df_category.append('nocoord')
                                name = name.replace('-opt-nocoord',"")
                            elif 'cocoord' in name:
                                category = 'cocoord'
                                df_category.append('cocoord')
                                name = name.replace('-opt-cocoord',"")
                            elif 'etcoord' in name:
                                category = 'etcoord'
                                df_category.append('etcoord') 
                                name = name.replace('-opt-etcoord',"")
                            else:
                                print('error, no category assigned')
                            df_filename.append(name)
                            print(name, category, natoms)

                            if name in checking_df['Name'].tolist():
                                to_check_df = checking_df[(checking_df['Name'] == name) & (checking_df['Category'] == category)]

                                if len(to_check_df) == 0:
                                    print('opt name not in spe list')
                                
                                elif natoms != to_check_df['NAtoms'].tolist()[0]:
                                    print('opt and spe file doesnt match')
                                    
                            else:
                                print('opt name not in spe list')
 

                            
delta_g_data = {'Name': df_filename, 'Category': df_category, 'Gibbs Free Energy Correction' : df_gibbs}      

descriptor_data = pd.DataFrame(delta_g_data)
descriptor_data.to_csv("output_deltaG.csv")                              
                            
                            # number_list = re.findall('-?\d*\.?\d+',line)            # get value
                            # for x in number_list:                                   # print value
                            #     print(phrase,': ', float(x))



n-n-1 etcoord 47
n-n-2 etcoord 50
n-n-3 etcoord 46
n-n-4 etcoord 64
n-n-5 etcoord 41
p-p-1 etcoord 71
p-p-2 etcoord 74
p-p-3 etcoord 90
p-p-4 etcoord 77
p-so-1 etcoord 64
p-so-10 etcoord 64
p-so-11 etcoord 74
p-so-12 etcoord 78
p-so-13 etcoord 62
p-so-14 etcoord 68
p-so-15 etcoord 54
p-so-16 etcoord 73
p-so-2 etcoord 56
p-so-3 etcoord 70
p-so-4 etcoord 76
p-so-5 etcoord 65
p-so-6 etcoord 60
p-so-7 etcoord 62
p-so-8 etcoord 72
p-so-9 etcoord 64
pcpo-1 etcoord 69
pcpo-2 etcoord 72
pcpo-3 etcoord 75
pnpo-1 etcoord 94
pnpo-10 etcoord 100
pnpo-11 etcoord 100
pnpo-12 etcoord 100
pnpo-13 etcoord 100
pnpo-2 etcoord 100
pnpo-3 etcoord 90
pnpo-4 etcoord 100
pnpo-5 etcoord 86
pnpo-6 etcoord 82
pnpo-7 etcoord 78
pnpo-8 etcoord 89
pnpo-9 etcoord 100
n-n-1 nocoord 41
n-n-2 nocoord 44
n-n-3 nocoord 40
n-n-4 nocoord 58
n-n-5 nocoord 35
p-p-1 nocoord 65
p-p-2 nocoord 68
p-p-3 nocoord 84
p-p-4 nocoord 71
p-so-1 nocoord 58
p-so-10 nocoord 58
p-so-11 nocoord 68
p-so-12 nocoord 72
p-so-13 nocoord 56
p-so-1

In [6]:
# Merge nbo and gibbs free energy dataset
# Calculate delta G difference between etcoord and nocoord


df_opt = pd.read_csv('output_deltaG.csv')
df_opt = df_opt.drop('Unnamed: 0',axis=1)

df_nbo = pd.read_csv('output_nbo.csv')
df_nbo = df_nbo.drop('Unnamed: 0',axis=1)

outer_merge = pd.merge(df_nbo,df_opt, how="outer", on=["Name", "Category"])
print(outer_merge.shape)
outer_merge.to_csv("output_merge.csv") 

(82, 16)


In [7]:
# Seperate dataframe into nocoord and etcoord

category_unique = outer_merge["Category"].unique()
for x in list(category_unique):
    df = outer_merge[outer_merge.Category == x]
    df.to_csv("df_" + x + ".csv") 

In [8]:
df_nocoord = pd.read_csv('df_nocoord.csv')
df_nocoord = df_nocoord.drop('Unnamed: 0',axis=1)
df_etcoord = pd.read_csv('df_etcoord.csv')
df_etcoord = df_etcoord.drop('Unnamed: 0',axis=1)
df_nocoord.rename(columns = {'SPE':'SPE_nocoord', 
                             'Gibbs Free Energy Correction':'Gibbs_Free_Energy_Correction_nocoord'}, inplace = True)
df_etcoord.rename(columns = {'SPE':'SPE_etcoord', 
                             'Gibbs Free Energy Correction':'Gibbs_Free_Energy_Correction_etcoord'}, inplace = True)
df_etcoord = df_etcoord[['Name','SPE_etcoord','Gibbs_Free_Energy_Correction_etcoord']]

new_merge = pd.merge(df_nocoord,df_etcoord, how="outer", on=["Name"])

# Add new column with the difference in Gibbs free energy between the etcoord and nocoord intermediates
new_merge['GFE_etcoord-nocoord'] = (627.5*(new_merge['Gibbs_Free_Energy_Correction_etcoord'] + new_merge['SPE_etcoord'] -
                                   (new_merge['Gibbs_Free_Energy_Correction_nocoord'] + new_merge['SPE_nocoord']) + 78.51996755))

# Remove columns which are not descriptors
new_merge = new_merge.drop(['Category',
                            'Metal',
                            'Atom_A',
                            'Atom_B',
                            'SPE_nocoord',
                            'SPE_etcoord',
                            'Gibbs_Free_Energy_Correction_nocoord',
                            'Gibbs_Free_Energy_Correction_etcoord'],axis=1)

# Merge with percent buried volume
df_buriedvol = pd.read_csv('percent_buried_volume.csv')
new_merge_2 = pd.merge(new_merge,df_buriedvol, how="outer", on=["Name"])

new_merge_2

,Name,A-M-B Angle,M-A Distance,M-B Distance,Metal_NBO,A_NBO,B_NBO,HOMO_LUMO,A-B_NBO,MA_MB_Difference,...,SW_VBur,NW_VBur,NE_VBur,SE_VBur,N_AVG,S_AVG,Phenylene_backbone,Pd-O Distance,Alkoxy,Charge
0,n-n-1,77,2.189937,2.069949,0.45674,-0.44516,-0.39852,0.13318,-0.04664,0.119988,...,34.2,34.7,51.9,34.2,43.30,34.20,1,0.000,0,1
1,n-n-2,77,2.170326,2.072112,0.49421,-0.41997,-0.39750,0.13923,-0.02247,0.098214,...,34.4,34.1,35.3,40.8,34.70,37.60,1,4.410,1,1
2,n-n-3,77,2.177374,2.074870,0.49349,-0.42170,-0.39797,0.15312,-0.02373,0.102504,...,33.9,34.7,33.6,41.2,34.15,37.55,1,0.000,0,1
3,n-n-4,77,2.166086,2.074475,0.48133,-0.43169,-0.39668,0.14574,-0.03501,0.091611,...,34.4,34.0,51.9,53.9,42.95,44.15,1,0.000,0,1
4,n-n-5,79,2.181435,2.081302,0.49330,-0.43370,-0.40167,0.16768,-0.03203,0.100133,...,34.3,33.9,31.8,31.7,32.85,33.00,1,0.000,0,1
5,p-p-1,85,2.421651,2.269837,0.02980,1.05375,1.24044,0.17512,-0.18669,0.151814,...,56.8,48.3,53.3,41.4,50.80,49.10,0,0.000,0,1
6,p-p-2,90,2.425881,2.277643,0.04242,1.04759,1.22754,0.17602,-0.17995,0.148238,...,49.8,54.9,52.9,49.4,53.90,49.60,0,0.000,0,1
7,p-p-3,92,2.441465,2.302020,0.16457,1.00995,1.18212,0.16375,-0.17217,0.139445,...,55.5,69.5,53.0,48.8,61.25,52.15,0,3.190,1,1
8,p-p-4,101,2.416731,2.269158,0.06011,1.03053,1.19386,0.15541,-0.16333,0.147573,...,49.5,53.4,50.2,44.6,51.80,47.05,1,0.000,0,1
9,p-so-1,92,2.455732,2.072498,0.38460,1.02659,-0.92891,0.16132,1.95550,0.383234,...,37.2,39.7,51.3,62.5,45.50,49.85,1,3.050,1,0


### Adding new steric descriptors from steric maps obtained through Sambvca
1. Obtain %Filled for each combination of axis/boundary/z-threshold
2. Correlate %Filled for each combination of axis/boundary/z-threshold with the nonalt_list to identify which combination would provide the highest correlation coefficient
3. Obtain boundary/z-threshold that has the highest correlation coefficient for the X & Y axis. 
4. Calculated weighted %Filled across various combinations of axis/boundary/z-threshold for each training data.


In [9]:
# Adding new steric descriptors from steric map raw data (%Filled_x, %Filled_y and weighted %filled)

# Import list of assigned nonalternating examples for measuring coefficient later
nonalt_list = pd.read_csv('nonalt_list.csv',header=None)
nonalt_list = nonalt_list.values.tolist()
nonalt_list = [val for sublist in nonalt_list for val in sublist]
    

#Set up new dataframe

data_dict = {'Name':[],
        'Axis':[],
        'Boundary':[],
        'Threshold':[],
        'Unfilled':[],
        'Filled':[],
        'Percent_Filled':[],
        'Nonalt':[]
       }

new_df = pd.DataFrame(data_dict)

        
for filename in os.listdir(steric_map_dir):
    csv_name = filename.replace(".csv","")
    name = csv_name.replace("-nocoord-map","")   #obtain catalyst name as a variable
    filepath = os.path.join(steric_map_dir,filename)
    print(name,filepath)
    dataset = pd.read_csv(filepath,header=None)
    dataset.columns = ["X-coord", "Y-coord", "Z"]

    # Set up datasets
    axis_list = ["X-coord",'Y-coord']
    min_list = [-1.00, -2.00, -3.00]
    max_list = [1.00, 2.00, 3.00]

    # Define upper and lower boundaries within the dataset
    for axis in axis_list: 
#         for min_boundary, max_boundary in zip(min_list, max_list):
#             df_overmin = dataset[dataset[axis] >= min_boundary]
#             df_lowermax = dataset[dataset[axis] <= max_boundary]
#             df_combined = pd.concat([df_overmin, df_lowermax])

#             threshold_list = np.arange(0,2,0.25)
#             boundary = str(min_boundary)+'-'+str(max_boundary)

#             for threshold in threshold_list:
#                 df_combined['Filled_Unfilled'] = np.where(df_combined['Z'] <= threshold, 'Unfilled', 'Filled')

#                 if name in nonalt_list:
#                     nonalt = 1
#                 else:
#                     nonalt = 0 
                
#                 print(df_combined.Filled_Unfilled.value_counts())

#                 try:
#                     unfilled = df_combined.Filled_Unfilled.value_counts().Unfilled
# #                     print('Unfilled: ', unfilled)
#                 except KeyError:
#                     unfilled = 0

#                 if unfilled == len(df_combined.index):
#                     filled_percent = 0
#                 else:
#                     filled = df_combined.Filled_Unfilled.value_counts().Filled
# #                     print('Filled: ', filled)
#                     filled_percent = 100 * filled / (unfilled + filled)
                    
        for min_boundary, max_boundary in zip(min_list,max_list):
            df_min = dataset[dataset[axis] >= min_boundary]
            df_minmax = df_min[df_min[axis] <= max_boundary]
            print(df_minmax)

    # Identify %Filled for each Z threshold based on voxel value

            threshold_list = list(np.arange(0,2,0.25))  # Define Z thresholds with 0.25 intervals
            boundary = str(min_boundary)+'-'+str(max_boundary)


            for threshold in threshold_list:
                df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
                df_minmax.loc[df_minmax['Z'] > threshold, 'Filled_Unfilled'] = 'Filled'

                if name in nonalt_list:
                    nonalt = 1
                else:
                    nonalt = 0 

                if df_minmax.Filled_Unfilled.value_counts().Unfilled == len(df_minmax.index):  # If all voxel is unfilled
                    filled = 0
                    unfilled = df_minmax.Filled_Unfilled.value_counts().Unfilled
                    filled_percent = 0

                else:
                    unfilled, filled = df_minmax['Filled_Unfilled'].value_counts()  # If some voxel is filled
                    filled_percent = 100 * filled/(unfilled+filled)

                new_row = {'Name':name,
                           'Axis':axis,
                           'Boundary':boundary,
                           'Threshold':threshold, 
                           'Unfilled':unfilled, 
                           'Filled':filled, 
                           'Percent_Filled':filled_percent,
                           'Nonalt':nonalt}
                new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)

#     #Add inversed dataset
#     for axis in axis_list: 
#         for min_boundary, max_boundary in zip(min_list, max_list):
#             df_overmax = dataset[dataset[axis] >= max_boundary]
#             df_lowermin = dataset[dataset[axis] <= min_boundary]
#             df_combined = pd.concat([df_overmax, df_lowermin])

#             threshold_list = np.linspace(0, 2, 9)
#             boundary = '-3.45 - '+str(min_boundary)+' and '+str(max_boundary) + ' - 3.55'

#             for threshold in threshold_list:
#                 df_combined['Filled_Unfilled'] = np.where(df_combined['Z'] <= threshold, 'Unfilled', 'Filled')

#                 nonalt = 1 if name in nonalt_list else 0
                
#                 print(df_combined.Filled_Unfilled.value_counts())

#                 try:
#                     unfilled = df_combined.Filled_Unfilled.value_counts().Unfilled
# #                     print('Unfilled: ', unfilled)
#                 except KeyError:
#                     unfilled = 0

#                 if unfilled == len(df_combined.index):
#                     filled_percent = 0
#                 else:
#                     filled = df_combined.Filled_Unfilled.value_counts().Filled
# #                     print('Filled: ', filled)
#                     filled_percent = 100 * filled / (unfilled + filled)




    for axis in axis_list:
        for min_boundary, max_boundary in zip(min_list,max_list):
            df_overmax = dataset[dataset[axis] >= max_boundary]                                #Inversed boundaries
            df_lowermin = dataset[dataset[axis] <= min_boundary]
            df_combined = pd.concat([df_overmax,df_lowermin])


           # Identify %Filled for each Z threshold

            threshold_list = list(np.arange(0,2,0.25))
            boundary = '-3.45 - '+str(min_boundary)+' and '+str(max_boundary) + ' - 3.55'

            for threshold in threshold_list:
                df_combined.loc[df_combined['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
                df_combined.loc[df_combined['Z'] > threshold, 'Filled_Unfilled'] = 'Filled'


                if name in nonalt_list:
                    nonalt = 1
                else:
                    nonalt = 0 


                if df_combined.Filled_Unfilled.value_counts().Unfilled == len(df_combined.index):
                    filled = 0
                    unfilled = df_combined.Filled_Unfilled.value_counts().Unfilled
                    filled_percent = 0

                else:
                    unfilled, filled = df_combined['Filled_Unfilled'].value_counts()
                    filled_percent = 100 * filled/(unfilled+filled)

                new_row = {'Name':name,
                           'Axis':axis,
                           'Boundary':boundary,
                           'Threshold':threshold, 
                           'Unfilled':unfilled, 
                           'Filled':filled, 
                           'Percent_Filled':filled_percent,
                           'Nonalt':nonalt}
                new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)

              
    
new_df = new_df.sort_values(by =['Axis', 'Boundary', 'Threshold'])
                

new_df.to_csv('steric_descriptors-training.csv')

n-n-1 C:\Users\George\Desktop\Research_UCLA\CIC\Computational Work\Project-3-2\intermediates\training_set\steric_maps_nocoord_training\n-n-1-nocoord-map.csv
      X-coord  Y-coord    Z
1775    -0.95    -3.45 -7.0
1776    -0.95    -3.35 -7.0
1777    -0.95    -3.25 -7.0
1778    -0.95    -3.15 -7.0
1779    -0.95    -3.05 -7.0
...       ...      ...  ...
3190     0.95     3.15 -7.0
3191     0.95     3.25 -7.0
3192     0.95     3.35 -7.0
3193     0.95     3.45 -7.0
3194     0.95     3.55 -7.0

[1420 rows x 3 columns]
      X-coord  Y-coord    Z
1065    -1.95    -3.45 -7.0
1066    -1.95    -3.35 -7.0
1067    -1.95    -3.25 -7.0
1068    -1.95    -3.15 -7.0
1069    -1.95    -3.05 -7.0
...       ...      ...  ...
3900     1.95     3.15 -7.0
3901     1.95     3.25 -7.0
3902     1.95     3.35 -7.0
3903     1.95     3.45 -7.0
3904     1.95     3.55 -7.0

[2840 rows x 3 columns]
      X-coord  Y-coord    Z
355     -2.95    -3.45 -7.0
356     -2.95    -3.35 -7.0
357     -2.95    -3.25 -7.0
358     -

C:\Users\George\AppData\Local\Temp\ipykernel_50740\2228760255.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_50740\2228760255.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_50740\2228760255.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy 

n-n-2 C:\Users\George\Desktop\Research_UCLA\CIC\Computational Work\Project-3-2\intermediates\training_set\steric_maps_nocoord_training\n-n-2-nocoord-map.csv
      X-coord  Y-coord    Z
1775    -0.95    -3.45 -7.0
1776    -0.95    -3.35 -7.0
1777    -0.95    -3.25 -7.0
1778    -0.95    -3.15 -7.0
1779    -0.95    -3.05 -7.0
...       ...      ...  ...
3190     0.95     3.15 -7.0
3191     0.95     3.25 -7.0
3192     0.95     3.35 -7.0
3193     0.95     3.45 -7.0
3194     0.95     3.55 -7.0

[1420 rows x 3 columns]
      X-coord  Y-coord    Z
1065    -1.95    -3.45 -7.0
1066    -1.95    -3.35 -7.0
1067    -1.95    -3.25 -7.0
1068    -1.95    -3.15 -7.0
1069    -1.95    -3.05 -7.0
...       ...      ...  ...
3900     1.95     3.15 -7.0
3901     1.95     3.25 -7.0
3902     1.95     3.35 -7.0
3903     1.95     3.45 -7.0
3904     1.95     3.55 -7.0

[2840 rows x 3 columns]
      X-coord  Y-coord    Z
355     -2.95    -3.45 -7.0
356     -2.95    -3.35 -7.0
357     -2.95    -3.25 -7.0
358     -

C:\Users\George\AppData\Local\Temp\ipykernel_50740\2228760255.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_50740\2228760255.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_50740\2228760255.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy 

n-n-3 C:\Users\George\Desktop\Research_UCLA\CIC\Computational Work\Project-3-2\intermediates\training_set\steric_maps_nocoord_training\n-n-3-nocoord-map.csv
      X-coord  Y-coord    Z
1775    -0.95    -3.45 -7.0
1776    -0.95    -3.35 -7.0
1777    -0.95    -3.25 -7.0
1778    -0.95    -3.15 -7.0
1779    -0.95    -3.05 -7.0
...       ...      ...  ...
3190     0.95     3.15 -7.0
3191     0.95     3.25 -7.0
3192     0.95     3.35 -7.0
3193     0.95     3.45 -7.0
3194     0.95     3.55 -7.0

[1420 rows x 3 columns]
      X-coord  Y-coord    Z
1065    -1.95    -3.45 -7.0
1066    -1.95    -3.35 -7.0
1067    -1.95    -3.25 -7.0
1068    -1.95    -3.15 -7.0
1069    -1.95    -3.05 -7.0
...       ...      ...  ...
3900     1.95     3.15 -7.0
3901     1.95     3.25 -7.0
3902     1.95     3.35 -7.0
3903     1.95     3.45 -7.0
3904     1.95     3.55 -7.0

[2840 rows x 3 columns]
      X-coord  Y-coord    Z
355     -2.95    -3.45 -7.0
356     -2.95    -3.35 -7.0
357     -2.95    -3.25 -7.0
358     -

...

C:\Users\George\AppData\Local\Temp\ipykernel_50740\2228760255.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'


pnpo-8 C:\Users\George\Desktop\Research_UCLA\CIC\Computational Work\Project-3-2\intermediates\training_set\steric_maps_nocoord_training\pnpo-8-nocoord-map.csv
      X-coord  Y-coord    Z
1775    -0.95    -3.45 -7.0
1776    -0.95    -3.35 -7.0
1777    -0.95    -3.25  0.2
1778    -0.95    -3.15  0.2
1779    -0.95    -3.05  0.2
...       ...      ...  ...
3190     0.95     3.15 -0.4
3191     0.95     3.25 -0.4
3192     0.95     3.35 -0.4
3193     0.95     3.45 -7.0
3194     0.95     3.55 -7.0

[1420 rows x 3 columns]
      X-coord  Y-coord    Z
1065    -1.95    -3.45 -7.0
1066    -1.95    -3.35 -7.0
1067    -1.95    -3.25 -7.0
1068    -1.95    -3.15 -7.0
1069    -1.95    -3.05 -7.0
...       ...      ...  ...
3900     1.95     3.15 -7.0
3901     1.95     3.25 -7.0
3902     1.95     3.35 -7.0
3903     1.95     3.45 -7.0
3904     1.95     3.55 -7.0

[2840 rows x 3 columns]
      X-coord  Y-coord    Z
355     -2.95    -3.45 -7.0
356     -2.95    -3.35 -7.0
357     -2.95    -3.25 -7.0
358    

C:\Users\George\AppData\Local\Temp\ipykernel_50740\2228760255.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_50740\2228760255.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_50740\2228760255.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy 

      X-coord  Y-coord    Z
15      -3.45    -1.95 -7.0
16      -3.45    -1.85 -7.0
17      -3.45    -1.75 -7.0
18      -3.45    -1.65 -7.0
19      -3.45    -1.55 -7.0
...       ...      ...  ...
5020     3.55     1.55 -7.0
5021     3.55     1.65 -7.0
5022     3.55     1.75 -7.0
5023     3.55     1.85 -7.0
5024     3.55     1.95 -7.0

[2840 rows x 3 columns]
      X-coord  Y-coord    Z
5       -3.45    -2.95 -7.0
6       -3.45    -2.85 -7.0
7       -3.45    -2.75 -7.0
8       -3.45    -2.65 -7.0
9       -3.45    -2.55 -7.0
...       ...      ...  ...
5030     3.55     2.55 -7.0
5031     3.55     2.65 -7.0
5032     3.55     2.75 -7.0
5033     3.55     2.85 -7.0
5034     3.55     2.95 -7.0

[4260 rows x 3 columns]
pnpo-9 C:\Users\George\Desktop\Research_UCLA\CIC\Computational Work\Project-3-2\intermediates\training_set\steric_maps_nocoord_training\pnpo-9-nocoord-map.csv
      X-coord  Y-coord    Z
1775    -0.95    -3.45 -7.0
1776    -0.95    -3.35 -7.0
1777    -0.95    -3.25 -0.3
1778   

C:\Users\George\AppData\Local\Temp\ipykernel_50740\2228760255.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_50740\2228760255.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minmax.loc[df_minmax['Z'] <= threshold, 'Filled_Unfilled'] = 'Unfilled'
C:\Users\George\AppData\Local\Temp\ipykernel_50740\2228760255.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [10]:
new_df.columns

Index(['Name', 'Axis', 'Boundary', 'Threshold', 'Unfilled', 'Filled',
       'Percent_Filled', 'Nonalt'],
      dtype='object')

In [11]:
# Obtaining coefficient for each axis/boundary/threshold combination

group_df = new_df.groupby(['Axis', 'Boundary','Threshold'])
# print(new_df.groupby(['Axis', 'Boundary','Threshold']).size())
# new_df.to_csv('group_df_steric_descriptors-training.csv')

coefficient_list = []
group_name_list = []
axis_list = []
boundary_list = []
threshold_list = []

x_coefficient = 0
y_coefficient = 0

for group_name, group_data in group_df:
#     print(group_df.get_group(key)) 
    axis_group_list = group_data["Axis"].values.tolist()
    axis_value = pd.unique(axis_group_list)
    axis_list.append(' '.join(axis_value))
    
    boundary_group_list = group_data["Boundary"].values.tolist()
#     print(boundary_group_list)
    boundary_value = pd.unique(boundary_group_list)
    boundary_list.append(' '.join(boundary_value))
    
    threshold_group_list = group_data["Threshold"].values.tolist()
#     print(threshold_group_list)
    threshold_value = pd.unique(threshold_group_list)
    threshold_value = str(threshold_value)[1:-1]
    threshold_list.append(threshold_value)

    if group_name[0] == 'X-coord':
        column_1 = group_data["Percent_Filled"]
        column_2 = group_data["Nonalt"]        
        coefficient = (column_1.corr(column_2))**2
        print('Name: ', group_name , 'Coefficient = %.2f' % coefficient)
        coefficient_list.append(coefficient)
        group_name_list.append(group_name)
        
        # Iterate through all combinations of boundaries/z-theshold until highest coefficient is obtained for the x-axis
        if coefficient > x_coefficient:
            x_coefficient = coefficient
            x_group_name = group_name
        
        
        
    elif group_name[0] == 'Y-coord':
        column_1 = group_data["Percent_Filled"]
        column_2 = group_data["Nonalt"]        
        coefficient = (column_1.corr(column_2))**2
        print('Name: ', group_name ,'Coefficient = %.2f' % coefficient)
        coefficient_list.append(coefficient)
        group_name_list.append(group_name)
        
        # Iterate through all combinations of boundaries/z-theshold until highest coefficient is obtained for the y-axis
        if coefficient > y_coefficient:
            y_coefficient = coefficient
            y_group_name = group_name

# Obtain combination of Axis, Boundary with highest coefficient for X & Y coordinates
print(x_group_name, 'Coefficient = %.2f' % x_coefficient)
print(y_group_name, 'Coefficient = %.2f' % y_coefficient)

#Store group combination into a txt file
with open('x_max_steric.txt','w') as file:
    file.write(str(x_group_name))
with open('y_max_steric.txt','w') as file:
    file.write(str(y_group_name))
    

#Obtain dataframe group with highest coefficient
x_max_steric = group_df.get_group(tuple(x_group_name))
print("x-max-steric: ", x_max_steric)
y_max_steric = group_df.get_group(tuple(y_group_name))
print("y-max-steric: ", y_max_steric)


#Obtain dataframe with group name & coefficient
coefficient_tuples = list(zip(axis_list, boundary_list, threshold_list, coefficient_list))
coefficient_df = pd.DataFrame(coefficient_tuples, columns=['Axis', 'Boundary','Threshold','Coefficient'])
coefficient_df.to_csv('coefficient_df.csv')


Name:  ('X-coord', '-1.0-1.0', 0.0) Coefficient = 0.16
Name:  ('X-coord', '-1.0-1.0', 0.25) Coefficient = 0.18
Name:  ('X-coord', '-1.0-1.0', 0.5) Coefficient = 0.23
Name:  ('X-coord', '-1.0-1.0', 0.75) Coefficient = 0.23
Name:  ('X-coord', '-1.0-1.0', 1.0) Coefficient = 0.23
Name:  ('X-coord', '-1.0-1.0', 1.25) Coefficient = 0.21
Name:  ('X-coord', '-1.0-1.0', 1.5) Coefficient = 0.19
Name:  ('X-coord', '-1.0-1.0', 1.75) Coefficient = 0.16
Name:  ('X-coord', '-2.0-2.0', 0.0) Coefficient = 0.09
Name:  ('X-coord', '-2.0-2.0', 0.25) Coefficient = 0.08
Name:  ('X-coord', '-2.0-2.0', 0.5) Coefficient = 0.19
Name:  ('X-coord', '-2.0-2.0', 0.75) Coefficient = 0.21
Name:  ('X-coord', '-2.0-2.0', 1.0) Coefficient = 0.25
Name:  ('X-coord', '-2.0-2.0', 1.25) Coefficient = 0.24
Name:  ('X-coord', '-2.0-2.0', 1.5) Coefficient = 0.21
Name:  ('X-coord', '-2.0-2.0', 1.75) Coefficient = 0.18
Name:  ('X-coord', '-3.0-3.0', 0.0) Coefficient = 0.01
Name:  ('X-coord', '-3.0-3.0', 0.25) Coefficient = 0.00
N

C:\Users\George\AppData\Local\Temp\ipykernel_50740\1512216370.py:19: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  axis_value = pd.unique(axis_group_list)
C:\Users\George\AppData\Local\Temp\ipykernel_50740\1512216370.py:24: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  boundary_value = pd.unique(boundary_group_list)
C:\Users\George\AppData\Local\Temp\ipykernel_50740\1512216370.py:29: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  threshold_value = pd.unique(threshold_group_list)


In [12]:
 
# printing correlation list
print("The original correlation list : " + str(coefficient_list))
 
# declaring magnitude of repetition with the number of training samples
K = len(new_merge_2)     
 
new_coefficient_list=[]
# repeat elements K times
for i in coefficient_list:
    new_coefficient_list.extend([i]*K)
    
# print("The list after adding elements : " + str(len(new_coefficient_list)))
new_df["Coefficient"] = new_coefficient_list

# Multiply each %Filled value with the correlation coefficient of the respective axis/boundary/z-threshold combination to obtain
# weighted %filled. Add into a new column in the dataframe
new_df['Weighted_percent_filled'] = new_df['Percent_Filled'] * new_df['Coefficient']
print(new_df)
new_df.to_csv('steric_descriptors-training-with-coefficient.csv')

The original correlation list : [0.16047289334578027, 0.1823863890080976, 0.22907103823105246, 0.22810387106491847, 0.2250532723860354, 0.20657424340670738, 0.1853601326980948, 0.15508879781746418, 0.09006697878834463, 0.07922380494302365, 0.188895148929098, 0.21369818957550524, 0.2472135063364244, 0.2424464402214507, 0.20579472898141377, 0.1775610729323613, 0.005759872630475843, 1.781187882284624e-05, 0.0476136119510626, 0.10856232989780552, 0.1626056579391367, 0.1809960421547085, 0.19991908237309355, 0.17445921638719342, 0.006328452441559489, 0.05466356095180827, 0.005739952313060874, 0.016014272638891942, 0.06623977211944854, 0.10207772843414145, 0.1422247455176173, 0.12914694522859685, 0.23255292638800648, 0.2425309374041493, 0.1093090433107694, 0.00824831512272701, 0.003124962592286457, 0.022079450680015638, 0.08938578488148997, 0.08728667075320586, 0.35382992955638515, 0.32201454504247506, 0.18427466358030115, 0.019194448087062676, 4.8201674782508056e-05, 0.021951332796353522, 0.

In [13]:
# Organize table according to the name of the training sample. 
# Obtain average of all weighted %filled values across various combinations of axis/boundary/z-threshold
group_df = new_df.groupby(['Name'])
name_list = []
average_weighted_percent_filled = []
for group_name, group_data in group_df:
    weighted_values = group_data["Weighted_percent_filled"]  
    name_list.append(group_name[0])
    average = (sum(weighted_values)/len(weighted_values))
    average_weighted_percent_filled.append(average)
#     print (group_name, average)

print(name_list)

avg_weighted_percent_filled_tuples = list(zip(name_list, average_weighted_percent_filled))
avg_weighted_percent_filled_df = pd.DataFrame(avg_weighted_percent_filled_tuples, columns=['Name','Weighted %Filled'])

print(avg_weighted_percent_filled_df)


['n-n-1', 'n-n-2', 'n-n-3', 'n-n-4', 'n-n-5', 'p-p-1', 'p-p-2', 'p-p-3', 'p-p-4', 'p-so-1', 'p-so-10', 'p-so-11', 'p-so-12', 'p-so-13', 'p-so-14', 'p-so-15', 'p-so-16', 'p-so-2', 'p-so-3', 'p-so-4', 'p-so-5', 'p-so-6', 'p-so-7', 'p-so-8', 'p-so-9', 'pcpo-1', 'pcpo-2', 'pcpo-3', 'pnpo-1', 'pnpo-10', 'pnpo-11', 'pnpo-12', 'pnpo-13', 'pnpo-2', 'pnpo-3', 'pnpo-4', 'pnpo-5', 'pnpo-6', 'pnpo-7', 'pnpo-8', 'pnpo-9']
       Name  Weighted %Filled
0     n-n-1          0.819442
1     n-n-2          0.660256
2     n-n-3          0.658078
3     n-n-4          1.012103
4     n-n-5          0.402397
5     p-p-1          1.037948
6     p-p-2          1.207724
7     p-p-3          2.255381
8     p-p-4          1.140081
9    p-so-1          1.421880
10  p-so-10          1.597753
11  p-so-11          2.183214
12  p-so-12          2.334004
13  p-so-13          0.500937
14  p-so-14          0.638684
15  p-so-15          0.613466
16  p-so-16          1.020122
17   p-so-2          0.358331
18   p-so-3      

In [14]:
x_max_steric

,Name,Axis,Boundary,Threshold,Unfilled,Filled,Percent_Filled,Nonalt
64,n-n-1,X-coord,-3.45 - -3.0 and 3.0 - 3.55,0.0,575.0,206.0,26.376440,0.0
160,n-n-2,X-coord,-3.45 - -3.0 and 3.0 - 3.55,0.0,575.0,206.0,26.376440,0.0
256,n-n-3,X-coord,-3.45 - -3.0 and 3.0 - 3.55,0.0,576.0,205.0,26.248399,0.0
352,n-n-4,X-coord,-3.45 - -3.0 and 3.0 - 3.55,0.0,559.0,222.0,28.425096,0.0
448,n-n-5,X-coord,-3.45 - -3.0 and 3.0 - 3.55,0.0,605.0,176.0,22.535211,0.0
544,p-p-1,X-coord,-3.45 - -3.0 and 3.0 - 3.55,0.0,613.0,168.0,21.510883,0.0
640,p-p-2,X-coord,-3.45 - -3.0 and 3.0 - 3.55,0.0,564.0,217.0,27.784891,0.0
736,p-p-3,X-coord,-3.45 - -3.0 and 3.0 - 3.55,0.0,541.0,240.0,30.729834,0.0
832,p-p-4,X-coord,-3.45 - -3.0 and 3.0 - 3.55,0.0,540.0,241.0,30.857875,0.0
928,p-so-1,X-coord,-3.45 - -3.0 and 3.0 - 3.55,0.0,645.0,136.0,17.413572,1.0


In [15]:
x_max_steric = x_max_steric.drop(['Axis','Boundary','Threshold','Unfilled','Filled','Nonalt'],axis=1)
x_max_steric = x_max_steric.rename(columns={"%Filled": "%Filled_x"})
y_max_steric = y_max_steric.drop(['Axis','Boundary','Threshold','Unfilled','Filled'],axis=1)
y_max_steric = y_max_steric.rename(columns={"%Filled": "%Filled_y"})


new_merge_3 = pd.merge(pd.merge(pd.merge(new_merge_2,x_max_steric, how="outer", on=["Name"])
                                ,y_max_steric, how="outer", on=["Name"])
                                ,avg_weighted_percent_filled_df, how="outer", on=["Name"])

new_merge_3.to_csv("all_descriptor_data.csv")  